# grg

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)
     

Thu Dec  8 04:10:58 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0    27W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


# gan

In [5]:
import glob

def preprocessing(input_path):
    all_input_list = sorted(glob.glob(input_path))
    train = pd.DataFrame()
    for datapath in all_input_list:
        data = pd.read_csv(datapath) 
  
        data['obs_time'] = data.index % 24 
        df = abs(data)
        df.loc[(df['내부온도관측치'] > 40), '내부온도관측치'] = 40
        df.loc[(df['내부습도관측치'] > 100), '내부습도관측치'] = 100
        df.loc[(df['co2관측치'] > 1200), 'co2관측치'] = 1200
        df.loc[(df['ec관측치'] > 8), 'ec관측치'] = 8
        df.loc[(df['시간당분무량'] > 3000), '시간당분무량'] = 3000
        df.loc[(df['시간당백색광량'] > 120000), '시간당백색광량'] = 120000
        df.loc[(df['시간당적색광량'] > 120000), '시간당적색광량'] = 120000
        df.loc[(df['시간당청색광량'] > 120000), '시간당청색광량'] = 120000
        df.loc[(df['시간당총광량'] > 120000), '시간당총광량'] = 120000
        df['시간당총광량'] = df['시간당청색광량']+df['시간당백색광량']+df['시간당적색광량']
        
        col_list = df.columns
        for i in range(0,len(col_list)):
            col = col_list[i]    
            if '누적' in col : 
                df[col] = df.groupby((df.obs_time == 0).cumsum()).agg(col_list[i-1]).cumsum()   
            df.to_csv(datapath,index=False)
            train = pd.concat([train,df])
    print('finish!!')
    return train


In [6]:
traininput = preprocessing('drive/MyDrive/dacon/gan/상추/train_input/*.csv')

finish!!


In [7]:
day0 = traininput[traininput['DAT']==0].reset_index(drop=True)
day0 = day0[['obs_time','내부온도관측치','내부습도관측치','co2관측치','ec관측치','시간당분무량','시간당백색광량','시간당적색광량','시간당청색광량']]
day0.describe()

,obs_time,내부온도관측치,내부습도관측치,co2관측치,ec관측치,시간당분무량,시간당백색광량,시간당적색광량,시간당청색광량
count,10752.000000,10752.000000,10752.000000,10752.000000,10752.000000,10752.000000,10752.000000,10752.000000,10752.000000
mean,11.500000,25.781004,54.912360,533.833783,1.273970,430.609479,6765.408458,1309.564887,856.852189
std,6.922508,4.361793,12.257845,144.127605,0.932456,491.308581,9450.283141,2653.722924,1938.172567
min,0.000000,0.000000,0.000000,60.400000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,5.750000,23.595463,49.183641,446.345833,0.401136,0.000000,0.000000,0.000000,0.000000
50%,11.500000,26.280460,56.510000,505.883333,1.212455,242.355000,0.000000,0.000000,0.000000
75%,17.250000,28.717917,61.310767,578.400000,2.011237,769.000000,18255.190000,976.332000,135.110625
max,23.000000,39.158823,81.900001,1200.000000,3.034100,2735.210000,120000.000000,9928.800000,18570.857500


In [8]:
day0

,obs_time,내부온도관측치,내부습도관측치,co2관측치,ec관측치,시간당분무량,시간당백색광량,시간당적색광량,시간당청색광량
0,0,25.300000,81.835000,536.016667,1.407439,0.0,0.0000,0.0,0.0
1,1,25.680357,81.264286,528.696429,1.409003,126.0,0.0000,0.0,0.0
2,2,25.273333,81.471666,532.833333,1.406913,0.0,0.0000,0.0,0.0
3,3,25.355000,81.398334,545.566667,1.406689,126.0,0.0000,0.0,0.0
4,4,25.391667,81.483333,558.583333,1.411070,0.0,0.0000,0.0,0.0
...,...,...,...,...,...,...,...,...,...
10747,19,29.980000,59.256667,505.466667,1.014238,0.0,9.2823,0.0,0.0
10748,20,29.730000,59.458333,504.433333,1.017222,0.0,0.0000,0.0,0.0
10749,21,29.491667,59.801667,501.216667,1.019655,769.0,0.0000,0.0,0.0
10750,22,29.531667,60.031667,501.400000,1.020756,0.0,0.0000,0.0,0.0


In [50]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
sc = MinMaxScaler()

In [51]:
sc.fit(day0)
newday0= sc.transform(day0)

In [11]:
from numpy import zeros
from numpy import ones
from numpy import hstack
from numpy.random import rand
from numpy.random import randn
from keras.models import Sequential
from keras.layers import Dense
from matplotlib import pyplot

In [85]:
import os
import pickle
import matplotlib.pyplot as plt
import torch
import torch.optim as optim 
from torch import nn
import pandas as pd
import numpy as np
from torch.optim import lr_scheduler

# 노이즈 -> 피쳐수만큼
# generator input -> 노이즈가 들어감
# generator ouput -> real data x 형태로 나와야 함.
# discriminator input -> real data x 가 들어감
# discriminator output -> 1

class Discriminator(nn.Module):
  def __init__(self):
    super(Discriminator, self).__init__()
    self.input = 9 
    self.output= 1
    self.model = nn.Sequential(
        nn.Linear(self.input, 16), # input size, hidden size
        nn.LeakyReLU(0.2),
        nn.Dropout(0.2),
        nn.Linear(16, 64),
        nn.LeakyReLU(0.8),
        nn.Linear(64, 128),
        nn.LeakyReLU(0.5),
        nn.Linear(128, self.output), # hidden size, output size
        nn.Sigmoid()
    ).to(device)

  def forward(self, x):
    #print(x.size())
    x = self.model(x)
    return x

class Generator(nn.Module):
  def __init__(self):
    super(Generator, self).__init__()
    self.features = 9 # 피쳐수
    self.output = 9  # 데이터수
    self.model = nn.Sequential(
        nn.Linear(self.features, 64), # input size, hidden size
        nn.Sigmoid(),
        nn.Dropout(0.2),
        nn.Linear(64, 16),
        #nn.LeakyReLU(0.2),
        nn.Linear(16,self.output), # hidden size, output size
        nn.Sigmoid()
    ).to(device)

  def forward(self, x):
    #print('generator',x.size())
    x = self.model(x)
    return x

# 모델 정의
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

generator = Generator().to(device)
discriminator = Discriminator().to(device)

g_optim = optim.Adam(generator.parameters(), lr=2e-4)
d_optim = optim.Adam(discriminator.parameters(), lr=2e-4)

criterion = nn.BCELoss().to(device)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer=g_optim, mode='min', verbose=True, patience=10, factor=0.5)

import time
n_epochs = 1000
noise = 9
start_time = time.time()
newdata = []
testdata = []
#print(real_data)
for epoch in range(n_epochs):

  test = day0.iloc[:28].values
  
  nptonn = torch.from_numpy(newday0).float()

  real = torch.cuda.FloatTensor(nptonn.size(0), 1).fill_(1.0) # 
  fake = torch.cuda.FloatTensor(nptonn.size(0), 1).fill_(0.0) # 

  real_data = nptonn.cuda()
  g_optim.zero_grad()

  z0 = torch.normal(mean=11.5, std=6.9, size=(nptonn.shape[0], 1)).cuda() # random noise
  z1 = torch.normal(mean=25.78, std=4.3, size=(nptonn.shape[0], 1)).cuda() # random noise
  z2 = torch.normal(mean=54.91, std=12.2, size=(nptonn.shape[0], 1)).cuda() # random noise
  z3 = torch.normal(mean=533.833, std=144.1, size=(nptonn.shape[0], 1)).cuda() # random noise
  z4 = torch.normal(mean=1.273, std=0.932, size=(nptonn.shape[0], 1)).cuda() # random noise
  z5 = torch.normal(mean=430.600, std=491.308, size=(nptonn.shape[0], 1)).cuda() # random noise
  z6 = torch.normal(mean=6765.408, std=9450.28, size=(nptonn.shape[0], 1)).cuda() # random noise
  z7 = torch.normal(mean=1309.564, std=2653.722, size=(nptonn.shape[0], 1)).cuda() # random noise
  z8 = torch.normal(mean=856.852, std=1938.17, size=(nptonn.shape[0], 1)).cuda() # random noise
  zz = torch.normal(mean=0, std=1, size=(nptonn.shape[0], noise)).cuda()

  z = torch.cat([z0,z1,z2,z3,z4,z5,z6,z7,z8], dim=1) #[M, N+N, K]
  #print(z.size())

  generated_dis = generator(z) # create distribution
  #print(generated_dis.size())
  #print(real_data.size())
  generated_dis_value = generated_dis.detach().cpu()
  g_loss =  criterion(discriminator(generated_dis), real) # calculate generator loss
  
  # update generator
  g_loss.backward()
  g_optim.step()

  # update discriminator
  real_loss = criterion(discriminator(real_data), real)
  r_score = discriminator(real_data).mean()
  fake_loss = criterion(discriminator(generated_dis.detach()), fake)
  g_score = discriminator(generated_dis).mean()
  d_loss = (real_loss + fake_loss) / 2

  newdata.append(generated_dis_value)

  d_loss.backward()
  d_optim.step()

  print(f"[Epoch {epoch}/{n_epochs}] [D loss: {d_loss.item():.6f}] [G loss: {g_loss.item():.6f}] [FAKE loss: {fake_loss.item():.6f}] ",
        g_score, r_score)

[Epoch 0/1000] [D loss: 0.695153] [G loss: 0.660646] [FAKE loss: 0.726829]  tensor(0.5165, device='cuda:0', grad_fn=<MeanBackward0>) tensor(0.5150, device='cuda:0', grad_fn=<MeanBackward0>)
[Epoch 1/1000] [D loss: 0.695528] [G loss: 0.653474] [FAKE loss: 0.734561]  tensor(0.5202, device='cuda:0', grad_fn=<MeanBackward0>) tensor(0.5187, device='cuda:0', grad_fn=<MeanBackward0>)
[Epoch 2/1000] [D loss: 0.695631] [G loss: 0.646730] [FAKE loss: 0.742016]  tensor(0.5239, device='cuda:0', grad_fn=<MeanBackward0>) tensor(0.5225, device='cuda:0', grad_fn=<MeanBackward0>)
[Epoch 3/1000] [D loss: 0.695816] [G loss: 0.639743] [FAKE loss: 0.749466]  tensor(0.5274, device='cuda:0', grad_fn=<MeanBackward0>) tensor(0.5261, device='cuda:0', grad_fn=<MeanBackward0>)
[Epoch 4/1000] [D loss: 0.696291] [G loss: 0.633179] [FAKE loss: 0.757242]  tensor(0.5311, device='cuda:0', grad_fn=<MeanBackward0>) tensor(0.5299, device='cuda:0', grad_fn=<MeanBackward0>)
[Epoch 5/1000] [D loss: 0.696843] [G loss: 0.62608

In [91]:
newdata[1][15]

tensor([0.4545, 0.4319, 0.5340, 0.5422, 0.4173, 0.5004, 0.4007, 0.5746, 0.4306])

In [88]:
newday0[15]

array([0.65217391, 0.72686896, 0.92376881, 0.39502165, 0.45635057,
       0.09213187, 0.15212658, 0.09833333, 0.        ])

In [92]:
import os
import pickle
import matplotlib.pyplot as plt
import torch
import torch.optim as optim 
from torch import nn
import pandas as pd
import numpy as np
from torch.optim import lr_scheduler

class Discriminator(nn.Module):
  def __init__(self):
    super(Discriminator, self).__init__()
    self.input = 9 
    self.output= 1
    self.model = nn.Sequential(
        nn.Linear(self.input, 32), # input size, hidden size
        nn.LeakyReLU(0.5),
        #nn.Dropout(0.2),
        nn.Linear(32, 64),
        nn.LeakyReLU(0.8),
        nn.Linear(64, 128),
        nn.LeakyReLU(0.5),
        nn.Linear(128, self.output), # hidden size, output size
        nn.Sigmoid()
    ).to(device)

  def forward(self, x):
    #print(x.size())
    x = self.model(x)
    return x

class Generator(nn.Module):
  def __init__(self):
    super(Generator, self).__init__()
    self.features = 9 # 피쳐수
    self.output = 9  # 데이터수
    self.model = nn.Sequential(
        nn.Linear(self.features, 64), # input size, hidden size
        nn.Sigmoid(),
        nn.Dropout(0.2),
        nn.Linear(64, 32),
        #nn.LeakyReLU(0.2),
        nn.Linear(32, self.output), # hidden size, output size
        nn.Sigmoid()
    ).to(device)

  def forward(self, x):
    #print('generator',x.size())
    x = self.model(x)
    return x

# 모델 정의
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

generator = Generator().to(device)
discriminator = Discriminator().to(device)

g_optim = optim.Adam(generator.parameters(), lr=2e-4)
d_optim = optim.Adam(discriminator.parameters(), lr=2e-4)

criterion = nn.BCELoss().to(device)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer=g_optim, mode='min', verbose=True, patience=10, factor=0.5)

import time
n_epochs = 1000
noise = 9
start_time = time.time()
newdata = []
testdata = []
#print(real_data)
for epoch in range(n_epochs):

  test = day0.iloc[:28].values
  
  nptonn = torch.from_numpy(newday0).float()

  real = torch.cuda.FloatTensor(nptonn.size(0), 1).fill_(1.0) # 
  fake = torch.cuda.FloatTensor(nptonn.size(0), 1).fill_(0.0) # 

  real_data = nptonn.cuda()
  g_optim.zero_grad()

  z0 = torch.normal(mean=11.5, std=6.9, size=(nptonn.shape[0], 1)).cuda() # random noise
  z1 = torch.normal(mean=25.78, std=4.3, size=(nptonn.shape[0], 1)).cuda() # random noise
  z2 = torch.normal(mean=54.91, std=12.2, size=(nptonn.shape[0], 1)).cuda() # random noise
  z3 = torch.normal(mean=533.833, std=144.1, size=(nptonn.shape[0], 1)).cuda() # random noise
  z4 = torch.normal(mean=1.273, std=0.932, size=(nptonn.shape[0], 1)).cuda() # random noise
  z5 = torch.normal(mean=430.600, std=491.308, size=(nptonn.shape[0], 1)).cuda() # random noise
  z6 = torch.normal(mean=6765.408, std=9450.28, size=(nptonn.shape[0], 1)).cuda() # random noise
  z7 = torch.normal(mean=1309.564, std=2653.722, size=(nptonn.shape[0], 1)).cuda() # random noise
  z8 = torch.normal(mean=856.852, std=1938.17, size=(nptonn.shape[0], 1)).cuda() # random noise
  zz = torch.normal(mean=0, std=1, size=(nptonn.shape[0], noise)).cuda()

  z = torch.cat([z0,z1,z2,z3,z4,z5,z6,z7,z8], dim=1) #[M, N+N, K]
  #print(z.size())

  generated_dis = generator(z) # create distribution
  #print(generated_dis.size())
  #print(real_data.size())
  generated_dis_value = generated_dis.detach().cpu()
  g_loss =  criterion(discriminator(generated_dis), real) # calculate generator loss
  
  # update generator
  g_loss.backward()
  g_optim.step()

  # update discriminator
  real_loss = criterion(discriminator(real_data), real)
  r_score = discriminator(real_data).mean()
  fake_loss = criterion(discriminator(generated_dis.detach()), fake)
  g_score = discriminator(generated_dis).mean()
  d_loss = (real_loss + fake_loss) / 2

  if g_score > 0.8 :
    newdata.append(generated_dis_value)

  d_loss.backward()
  d_optim.step()

  print(f"[Epoch {epoch}/{n_epochs}] [D loss: {d_loss.item():.6f}] [G loss: {g_loss.item():.6f}] [FAKE loss: {fake_loss.item():.6f}] ",
        g_score, r_score)

[Epoch 0/1000] [D loss: 0.698281] [G loss: 0.717518] [FAKE loss: 0.669360]  tensor(0.4880, device='cuda:0', grad_fn=<MeanBackward0>) tensor(0.4833, device='cuda:0', grad_fn=<MeanBackward0>)
[Epoch 1/1000] [D loss: 0.698361] [G loss: 0.711011] [FAKE loss: 0.675600]  tensor(0.4911, device='cuda:0', grad_fn=<MeanBackward0>) tensor(0.4862, device='cuda:0', grad_fn=<MeanBackward0>)
[Epoch 2/1000] [D loss: 0.698465] [G loss: 0.704787] [FAKE loss: 0.681645]  tensor(0.4942, device='cuda:0', grad_fn=<MeanBackward0>) tensor(0.4891, device='cuda:0', grad_fn=<MeanBackward0>)
[Epoch 3/1000] [D loss: 0.698607] [G loss: 0.698636] [FAKE loss: 0.687691]  tensor(0.4973, device='cuda:0', grad_fn=<MeanBackward0>) tensor(0.4919, device='cuda:0', grad_fn=<MeanBackward0>)
[Epoch 4/1000] [D loss: 0.698783] [G loss: 0.692512] [FAKE loss: 0.693785]  tensor(0.5003, device='cuda:0', grad_fn=<MeanBackward0>) tensor(0.4947, device='cuda:0', grad_fn=<MeanBackward0>)
[Epoch 5/1000] [D loss: 0.698999] [G loss: 0.68637

In [97]:
newdata[1][20]

tensor([0.6706, 0.5291, 0.4776, 0.7575, 0.4628, 0.7214, 0.5276, 0.3733, 0.4033])

In [98]:
newday0[20]

array([0.86956522, 0.69230885, 0.97745217, 0.38405288, 0.45395526,
       0.04371511, 0.00603349, 0.00466667, 0.        ])

In [114]:
import os
import pickle
import matplotlib.pyplot as plt
import torch
import torch.optim as optim 
from torch import nn
import pandas as pd
import numpy as np
from torch.optim import lr_scheduler

class Discriminator(nn.Module):
  def __init__(self):
    super(Discriminator, self).__init__()
    self.input = 9 
    self.output= 1
    self.model = nn.Sequential(
        nn.Linear(self.input, 32), # input size, hidden size
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(32, 64),
        nn.LeakyReLU(0.8),
        nn.Linear(64, 128),
        nn.LeakyReLU(0.5),
        nn.Linear(128, self.output), # hidden size, output size
        nn.Sigmoid()
    ).to(device)

  def forward(self, x):
    #print(x.size())
    x = self.model(x)
    return x

class Generator(nn.Module):
  def __init__(self):
    super(Generator, self).__init__()
    self.features = 9 # 피쳐수
    self.output = 9  # 데이터수
    self.model = nn.Sequential(
        nn.Linear(self.features, 128), # input size, hidden size
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(128, 32),
        nn.Sigmoid(),
        #nn.LeakyReLU(0.2),
        nn.Linear(32, self.output), # hidden size, output size
        nn.Sigmoid()
    ).to(device)

  def forward(self, x):
    #print('generator',x.size())
    x = self.model(x)
    return x

# 모델 정의
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

generator = Generator().to(device)
discriminator = Discriminator().to(device)

g_optim = optim.Adam(generator.parameters(), lr=2e-4)
d_optim = optim.Adam(discriminator.parameters(), lr=2e-4)

criterion = nn.BCELoss().to(device)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer=g_optim, mode='min', verbose=True, patience=10, factor=0.5)

import time
n_epochs = 1000
noise = 9
start_time = time.time()
newdata = []
testdata = []
#print(real_data)
for epoch in range(n_epochs):

  test = day0.iloc[:28].values
  
  nptonn = torch.from_numpy(newday0).float()

  real = torch.cuda.FloatTensor(nptonn.size(0), 1).fill_(1.0) # 
  fake = torch.cuda.FloatTensor(nptonn.size(0), 1).fill_(0.0) # 

  real_data = nptonn.cuda()
  g_optim.zero_grad()

  z0 = torch.normal(mean=11.5, std=6.9, size=(nptonn.shape[0], 1)).cuda() # random noise
  z1 = torch.normal(mean=25.78, std=4.3, size=(nptonn.shape[0], 1)).cuda() # random noise
  z2 = torch.normal(mean=54.91, std=12.2, size=(nptonn.shape[0], 1)).cuda() # random noise
  z3 = torch.normal(mean=533.833, std=144.1, size=(nptonn.shape[0], 1)).cuda() # random noise
  z4 = torch.normal(mean=1.273, std=0.932, size=(nptonn.shape[0], 1)).cuda() # random noise
  z5 = torch.normal(mean=430.600, std=491.308, size=(nptonn.shape[0], 1)).cuda() # random noise
  z6 = torch.normal(mean=6765.408, std=9450.28, size=(nptonn.shape[0], 1)).cuda() # random noise
  z7 = torch.normal(mean=1309.564, std=2653.722, size=(nptonn.shape[0], 1)).cuda() # random noise
  z8 = torch.normal(mean=856.852, std=1938.17, size=(nptonn.shape[0], 1)).cuda() # random noise
  zz = torch.normal(mean=0, std=1, size=(nptonn.shape[0], noise)).cuda()

  z = torch.cat([z0,z1,z2,z3,z4,z5,z6,z7,z8], dim=1) #[M, N+N, K]
  #print(z.size())

  generated_dis = generator(z) # create distribution
  #print(generated_dis.size())
  #print(real_data.size())
  generated_dis_value = generated_dis.detach().cpu()
  g_loss =  criterion(discriminator(generated_dis), real) # calculate generator loss
  
  # update generator
  g_loss.backward()
  g_optim.step()

  # update discriminator
  real_loss = criterion(discriminator(real_data), real)
  r_score = discriminator(real_data).mean()
  fake_loss = criterion(discriminator(generated_dis.detach()), fake)
  g_score = discriminator(generated_dis).mean()
  d_loss = (real_loss + fake_loss) / 2

  if g_score > 0.8 :
    newdata.append(generated_dis_value)

  d_loss.backward()
  d_optim.step()

  print(f"[Epoch {epoch}/{n_epochs}] [D loss: {d_loss.item():.6f}] [G loss: {g_loss.item():.6f}] [FAKE loss: {fake_loss.item():.6f}] ",
        g_score, r_score)

[Epoch 0/1000] [D loss: 0.699701] [G loss: 0.608103] [FAKE loss: 0.786441]  tensor(0.5445, device='cuda:0', grad_fn=<MeanBackward0>) tensor(0.5415, device='cuda:0', grad_fn=<MeanBackward0>)
[Epoch 1/1000] [D loss: 0.700927] [G loss: 0.601284] [FAKE loss: 0.794602]  tensor(0.5482, device='cuda:0', grad_fn=<MeanBackward0>) tensor(0.5449, device='cuda:0', grad_fn=<MeanBackward0>)
[Epoch 2/1000] [D loss: 0.702200] [G loss: 0.594715] [FAKE loss: 0.802637]  tensor(0.5517, device='cuda:0', grad_fn=<MeanBackward0>) tensor(0.5479, device='cuda:0', grad_fn=<MeanBackward0>)
[Epoch 3/1000] [D loss: 0.703381] [G loss: 0.588613] [FAKE loss: 0.810652]  tensor(0.5553, device='cuda:0', grad_fn=<MeanBackward0>) tensor(0.5509, device='cuda:0', grad_fn=<MeanBackward0>)
[Epoch 4/1000] [D loss: 0.704363] [G loss: 0.581906] [FAKE loss: 0.818443]  tensor(0.5589, device='cuda:0', grad_fn=<MeanBackward0>) tensor(0.5540, device='cuda:0', grad_fn=<MeanBackward0>)
[Epoch 5/1000] [D loss: 0.705943] [G loss: 0.57560

In [113]:
newdata[1][2]

tensor([0.5201, 0.5828, 0.5117, 0.6643, 0.4171, 0.4135, 0.3808, 0.6583, 0.5251])